In [11]:
# Use Pandas to read your cleaned measurements and stations CSV data.
#import dependencies
import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine, MetaData

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Use Pandas to read in the measurement and station CSV files as DataFrames.
measurements_to_load = "clean_hawaii_measurements.csv"
stations_to_load = "clean_hawaii_stations.csv"

# Read in the measurements and stations data
clean_hawaii_measurements = pd.read_csv(measurements_to_load)
clean_hawaii_stations = pd.read_csv(stations_to_load)

clean_hawaii_measurements.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-06,0.01,73


In [12]:
# Use the `engine` and connection string to create a database called `hawaii.sqlite`.
engine = create_engine("sqlite:///hawaii.sqlite")
conn = engine.connect()

In [13]:
# Use `declarative_base` and create ORM classes for each table.
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [14]:
# You will need a class for `Measurement` and for `Station`.

# Make sure to define your primary keys.
class Measurement(Base):
    __tablename__='Measurements'
    
    id = Column(Integer, primary_key = True)
    station = Column(String(20))
    date = Column(String(20))
    prcp = Column(Float)
    tobs = Column(Integer)
    
class Station(Base):
    __tablename__='Stations'
    
    id = Column(Integer, primary_key = True)
    station = Column(String(20))
    name = Column(String(20))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f'id={self.id}, name={self.name}'
    
# Once you have your ORM classes defined, 
# create the tables in the database using `create_all`.
Base.metadata.create_all(engine)

In [15]:
# Create list of data to write into tables
measurements_data = clean_hawaii_measurements.to_dict(orient="records")
stations_data = clean_hawaii_stations.to_dict(orient="records")


In [23]:
# Print measurements data in list
print(measurements_data[:10])

[{'Unnamed: 0': 0, 'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'Unnamed: 0': 1, 'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'Unnamed: 0': 2, 'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'Unnamed: 0': 3, 'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'Unnamed: 0': 4, 'station': 'USC00519397', 'date': '2010-01-06', 'prcp': 0.01, 'tobs': 73}, {'Unnamed: 0': 5, 'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}, {'Unnamed: 0': 6, 'station': 'USC00519397', 'date': '2010-01-08', 'prcp': 0.0, 'tobs': 64}, {'Unnamed: 0': 7, 'station': 'USC00519397', 'date': '2010-01-09', 'prcp': 0.0, 'tobs': 68}, {'Unnamed: 0': 8, 'station': 'USC00519397', 'date': '2010-01-10', 'prcp': 0.0, 'tobs': 73}, {'Unnamed: 0': 9, 'station': 'USC00519397', 'date': '2010-01-11', 'prcp': 0.01, 'tobs': 64}]


In [24]:
# print stations data in list
print(stations_data[:10])

[{'Unnamed: 0': 0, 'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'Unnamed: 0': 1, 'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'Unnamed: 0': 2, 'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'Unnamed: 0': 3, 'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'Unnamed: 0': 4, 'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}, {'Unnamed: 0': 5, 'station': 'USC00519523', 'name': 'WAIMANALO EXPERIMENTAL FARM, HI US', 'latitude': 21.33556, 'longitude': -157.71139, 'elevation': 19.5}, {'Unnamed: 0': 6, 'station': 'USC00519281', 'name': 'WAIHEE 837.5, HI US', 'latitude': 21.45167, 'longitude': -157.84

In [25]:
# Reflect tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [26]:
#Save references to variables
measurements_table = sqlalchemy.Table("measurements", metadata, autoload=True)
stations_table = sqlalchemy.Table("stations", metadata, autoload=True)

In [27]:
# Insert data into tables
conn.execute(measurements_table.insert(), measurements_data)
conn.execute(stations_table.insert(), stations_data)

In [28]:
# View measurements data in tables
conn.execute("select * from measurements limit 10").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.01, 73),
 (6, 'USC00519397', '2010-01-07', 0.06, 70),
 (7, 'USC00519397', '2010-01-08', 0.0, 64),
 (8, 'USC00519397', '2010-01-09', 0.0, 68),
 (9, 'USC00519397', '2010-01-10', 0.0, 73),
 (10, 'USC00519397', '2010-01-11', 0.01, 64)]

In [29]:
# View stations data in tables
conn.execute("select * from stations limit 10").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4),
 (10, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0)]